In [0]:
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2025/01/01")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2025/01/02")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2025/01/03")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2025/01/04")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2025/01/05")

In [0]:
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/autoloader_input/2025/01/01")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/autoloader_input/2025/01/02")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv", "/Volumes/dev/bronze/landing/autoloader_input/2025/01/03")

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-05.csv", "/Volumes/dev/bronze/landing/autoloader_input/2025/01/05")

In [0]:
df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format","csv")
    .option("pathGlobFilter","*.csv")
    .option("header", "true")
    .option("cloudFiles.schemaHints", "Quantity int, UnitPrice double")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)


In [0]:
from pyspark.sql.functions import *

In [0]:
(
    df
    .withColumn("_file", col("_metadata.file_name"))
    .writeStream
    .option("checkpointLocation","/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .outputMode("append")
    .trigger(availableNow = True)
    .toTable(("dev.bronze.invoice_a1_1"))
    )

In [0]:
%sql
select * from dev.bronze.invoice_a1_1;


In [0]:
%sql
select _file, count(1)
from dev.bronze.invoice_a1_1
group by _file

In [0]:
## auto loader begahaviour on reading new column

## addNewColumn(default) -- Stream fails
## rescue  -- schema never evolved and stream does ot fail due to scheam changes
## failOnNewColumns  --stream fails
## None --new column are ignores

In [0]:
df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format","csv")
    .option("pathGlobFilter","*.csv")
    .option("header", "true")
    .option("cloudFiles.useNotifications", "true")
    .option("cloudFiles.schemaHints", "Quantity int, UnitPrice double")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .option("schenmaevolutionMode", "None")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)